## Setup

Import required libraries and configure the base URL.

In [ ]:
import requests
import json
from pprint import pprint

# Configure your base URL
# For local FastAPI server:
BASE_URL = "http://localhost:8000/api/v1"

# For Domino deployment (update with your Domino URL):
# DOMINO_URL = "https://domino.web.bms.com:8888/your-project-path"

# Timeout for requests
TIMEOUT = 30

# Common headers
HEADERS = {
    "Content-Type": "application/json"
}

print("Setup complete!")
print(f"Base URL: {BASE_URL}")

## Health Check Endpoints

### 1. Health Check

In [ ]:
# Health check endpoint
url = f"{BASE_URL}/health"

response = requests.get(url, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 2. AEM Health Check

In [ ]:
# AEM health check endpoint
url = f"{BASE_URL}/aem"

response = requests.get(url, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

## Sites Endpoints

### 3. List Pages

In [ ]:
# List pages endpoint
url = f"{BASE_URL}/sites/list-pages"

payload = {
    "site_path": "/content/commercial/mava-international"
}

response = requests.post(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 4. Duplicate Template

In [ ]:
# Duplicate template endpoint
url = f"{BASE_URL}/sites/duplicate-template"

payload = {
    "market_region": "india",  # Required: Market region (e.g., 'india', 'germany', 'usa', 'uk', 'france')
    "source_path": "/content/buildeasy/mava/hcp-template"  # Required: Source template path to duplicate from
}

response = requests.post(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 4b. Copy Site Directly (AEM Site Copy)

In [ ]:
# Direct AEM site copy using AEM's site copy endpoint
# This bypasses the FastAPI endpoint and calls AEM directly

import requests
from requests.auth import HTTPBasicAuth

# AEM credentials (update these with your credentials)
AEM_HOST = "https://aem-dev-author.bms.com"
AEM_USERNAME = "your_username"
AEM_PASSWORD = "your_password"

# Source and destination paths
source_path = "/content/buildeasy/mava/hcp-template"
destination_path = "/content/buildeasy/mava/hcp-india-copy"

# Step 1: Get CSRF token
csrf_url = f"{AEM_HOST}/libs/granite/csrf/token.json"
csrf_response = requests.get(
    csrf_url,
    auth=HTTPBasicAuth(AEM_USERNAME, AEM_PASSWORD),
    verify=False
)

csrf_token = csrf_response.json().get("token")
print(f"CSRF Token: {csrf_token[:20]}..." if csrf_token else "No token")

# Step 2: Execute site copy operation using site.copy.json endpoint
copy_url = f"{AEM_HOST}/bin/wcm/siteadmin/site.copy.json"

# Site copy data
copy_data = {
    "src": source_path,
    "dest": destination_path,
    "async": "true"  # Asynchronous copy
}

headers = {
    "CSRF-Token": csrf_token,
    "Referer": AEM_HOST
}

copy_response = requests.post(
    copy_url,
    data=copy_data,
    headers=headers,
    auth=HTTPBasicAuth(AEM_USERNAME, AEM_PASSWORD),
    verify=False,
    timeout=60
)

print(f"\nSite Copy Operation Status Code: {copy_response.status_code}")

if copy_response.status_code in [200, 201]:
    response_data = copy_response.json()
    job_id = response_data.get("jobId")
    print(f"Response: {response_data}")
    print(f"\n✓ Site copy initiated successfully!")
    print(f"Job ID: {job_id}")
    print(f"Source: {source_path}")
    print(f"Destination: {destination_path}")
    print(f"\nNote: Copy is running asynchronously. Check AEM for completion status.")
else:
    print(f"\n✗ Site copy operation failed")
    print(f"Response: {copy_response.text}")

## Content Endpoints - Error Pages

### 5. Create Error Pages

In [ ]:
# Create error pages endpoint
url = f"{BASE_URL}/content/error-pages"

payload = {
    "page_path_404": "/content/buildeasy/mava/hcp-india/en_in/404",
    "page_path_500": "/content/buildeasy/mava/hcp-india/en_in/500",
    "jcr_content_404": {
        "jcr:title": "Page Not Found - 404",
        "jcr:description": "The page you are looking for could not be found",
        "errorCode": "404"
    },
    "jcr_content_500": {
        "jcr:title": "Server Error - 500",
        "jcr:description": "An internal server error occurred",
        "errorCode": "500"
    }
}

response = requests.post(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 6. Get Error Pages

In [ ]:
# Get error pages endpoint
url = f"{BASE_URL}/content/error-pages"

payload = {
    "page_path_404": "/content/buildeasy/mava/hcp-india/en_in/404",
    "page_path_500": "/content/buildeasy/mava/hcp-india/en_in/500"
}

response = requests.get(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 6b. Update Error Page Rich Text Component (Direct AEM)

In [ ]:
# Update error page rich text component directly in AEM
# This updates the text content in the existing rich text component

import requests
from requests.auth import HTTPBasicAuth

# AEM credentials (update these with your credentials)
AEM_HOST = "https://aem-dev-author.bms.com"
AEM_USERNAME = "your_username"
AEM_PASSWORD = "your_password"

# Error page path (choose 404 or 500)
error_page_path = "/content/buildeasy/mava/hcp-india/en_in/500"

# Step 1: Get CSRF token
csrf_url = f"{AEM_HOST}/libs/granite/csrf/token.json"
csrf_response = requests.get(
    csrf_url,
    auth=HTTPBasicAuth(AEM_USERNAME, AEM_PASSWORD),
    verify=False
)

csrf_token = csrf_response.json().get("token")
print(f"CSRF Token: {csrf_token[:20]}..." if csrf_token else "No token")

headers = {
    "CSRF-Token": csrf_token,
    "Referer": AEM_HOST,
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
}

# Step 2: Update the first rich text component (inside bgcolorcontainer)
# Using .html extension tells AEM to update the node properties
rich_text_path_1 = f"{error_page_path}/jcr:content/responsivegrid/responsive_column_co/1colpar1/bgcolorcontainer/par/rich_text.html"

# New HTML content for the first rich text component
update_data_1 = {
    "text": '<h1 style="text-align: center;"><b><span class="text-large">Service Temporarily Unavailable - Updated!</span></b></h1>\n',
    "textIsRich": "true"
}

# Update first rich text component
update_response_1 = requests.post(
    f"{AEM_HOST}{rich_text_path_1}",
    data=update_data_1,
    headers=headers,
    auth=HTTPBasicAuth(AEM_USERNAME, AEM_PASSWORD),
    verify=False,
    timeout=30
)

print(f"\nFirst Rich Text Component Update Status: {update_response_1.status_code}")
print(f"Response: {update_response_1.text[:500]}")

if update_response_1.status_code in [200, 201]:
    print(f"\n✓ First rich text component updated successfully!")
else:
    print(f"\n✗ First update failed")

# Step 3: Update the second rich text component
rich_text_path_2 = f"{error_page_path}/jcr:content/responsivegrid/responsive_column_co/1colpar1/rich_text_646410301.html"

# New HTML content for the second rich text component
update_data_2 = {
    "text": '<p style="text-align: left;">We apologize for the inconvenience. Our team is working to restore service.</p>\r\n<p style="text-align: left;">For urgent matters, please contact Bristol Myers Squibb Medical Information:</p>\r\n<p style="text-align: left;">Phone: 1-800-XXX-XXXX</p>\r\n<p style="text-align: left;">Email: medical.info@bms.com</p>\r\n',
    "textIsRich": "true"
}

# Update second rich text component
update_response_2 = requests.post(
    f"{AEM_HOST}{rich_text_path_2}",
    data=update_data_2,
    headers=headers,
    auth=HTTPBasicAuth(AEM_USERNAME, AEM_PASSWORD),
    verify=False,
    timeout=30
)

print(f"\nSecond Rich Text Component Update Status: {update_response_2.status_code}")
print(f"Response: {update_response_2.text[:500]}")

if update_response_2.status_code in [200, 201]:
    print(f"\n✓ Second rich text component updated successfully!")
else:
    print(f"\n✗ Second update failed")

print(f"\n{'='*60}")
print(f"Summary:")
print(f"Page: {error_page_path}")
print(f"Rich Text 1 (Header): {'✓ Updated' if update_response_1.status_code in [200, 201] else '✗ Failed'}")
print(f"Rich Text 2 (Contact Info): {'✓ Updated' if update_response_2.status_code in [200, 201] else '✗ Failed'}")
print(f"{'='*60}")

print(f"\n💡 Key Changes Made:")
print(f"   - Added .html extension to node path (Sling POST servlet)")
print(f"   - Removed './' prefix from property names")
print(f"   - Changed Content-Type header to application/x-www-form-urlencoded")
print(f"   - This approach directly updates node properties without creating children")

## Content Endpoints - Protected Pages

### 7. Create Protected Page

In [ ]:
# Create protected page endpoint
url = f"{BASE_URL}/content/protected-page"

payload = {
    "site_path": "/content/commercial/mava-international",
    "page_title": "HCP Only Content",
    "page_name": "hcp-content",
    "template_path": "/apps/commercial/templates/protected-page"
}

response = requests.post(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 8. Get Protected Page

In [ ]:
# Get protected page endpoint
url = f"{BASE_URL}/content/protected-page"

payload = {
    "page_path": "/content/commercial/mava-international/hcp-content"
}

response = requests.get(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

## Content Endpoints - HCP Modal Popup

### 9. Create HCP Modal Popup

In [ ]:
# Create HCP modal popup endpoint
url = f"{BASE_URL}/content/hcp-modal-popup"

payload = {
    "site_path": "/content/commercial/mava-international",
    "popup_title": "Healthcare Professional Verification",
    "popup_content": "This content is intended for healthcare professionals only.",
    "component_path": "/apps/commercial/components/hcp-popup"
}

response = requests.post(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 10. Get HCP Modal Popup

In [ ]:
# Get HCP modal popup endpoint
url = f"{BASE_URL}/content/hcp-modal-popup"

payload = {
    "site_path": "/content/commercial/mava-international"
}

response = requests.get(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

## Content Endpoints - Login Pages

### 11. Create Login Page

In [ ]:
# Create login page endpoint
url = f"{BASE_URL}/content/login-page"

payload = {
    "site_path": "/content/commercial/mava-international",
    "page_title": "HCP Login",
    "page_name": "login",
    "template_path": "/apps/commercial/templates/login-page",
    "redirect_url": "/content/commercial/mava-international/hcp-content"
}

response = requests.post(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

### 12. Get Login Page

In [ ]:
# Get login page endpoint
url = f"{BASE_URL}/content/login-page"

payload = {
    "page_path": "/content/commercial/mava-international/login"
}

response = requests.get(url, json=payload, headers=HEADERS, timeout=TIMEOUT)

print(f"Status Code: {response.status_code}")
pprint(response.json())

## Domino Function Invocation

Test the `get_app()` function for Domino deployment.

### 13. Domino - List Pages

In [ ]:
# For Domino deployment - update DOMINO_URL first
# DOMINO_URL = "https://domino.web.bms.com:8888/your-project-path"

# Domino payload format
domino_payload = {
    "endpoint": "list-pages",
    "request_body": {
        "site_path": "/content/commercial/mava-international"
    }
}

# Uncomment to test with Domino
# response = requests.post(DOMINO_URL, json=domino_payload, headers=HEADERS, timeout=TIMEOUT, verify=False)
# print(f"Status Code: {response.status_code}")
# pprint(response.json())

print("Domino payload prepared:")
pprint(domino_payload)

### 14. Domino - Health Check

In [ ]:
# Domino health check
domino_payload = {
    "endpoint": "health",
    "request_body": {}
}

# Uncomment to test with Domino
# response = requests.post(DOMINO_URL, json=domino_payload, headers=HEADERS, timeout=TIMEOUT, verify=False)
# print(f"Status Code: {response.status_code}")
# pprint(response.json())

print("Domino payload prepared:")
pprint(domino_payload)

### 15. Domino - Create Error Pages

In [ ]:
# Domino create error pages
domino_payload = {
    "endpoint": "create-error-pages",
    "request_body": {
        "page_path_404": "/content/buildeasy/mava/hcp-india/en_in/404",
        "page_path_500": "/content/buildeasy/mava/hcp-india/en_in/500",
        "jcr_content_404": {
            "jcr:title": "Page Not Found - 404",
            "jcr:description": "The page you are looking for could not be found",
            "errorCode": "404"
        },
        "jcr_content_500": {
            "jcr:title": "Server Error - 500",
            "jcr:description": "An internal server error occurred",
            "errorCode": "500"
        }
    }
}

# Uncomment to test with Domino
# response = requests.post(DOMINO_URL, json=domino_payload, headers=HEADERS, timeout=TIMEOUT, verify=False)
# print(f"Status Code: {response.status_code}")
# pprint(response.json())

print("Domino payload prepared:")
pprint(domino_payload)

## Utility Functions

### Helper Function to Test All Endpoints

In [ ]:
def test_all_endpoints():
    """Test all available endpoints and show summary"""
    
    endpoints = [
        {"name": "Health Check", "method": "GET", "url": f"{BASE_URL}/health", "payload": None},
        {"name": "AEM Health", "method": "GET", "url": f"{BASE_URL}/aem", "payload": None},
        {
            "name": "List Pages",
            "method": "POST",
            "url": f"{BASE_URL}/sites/list-pages",
            "payload": {"site_path": "/content/commercial/mava-international"}
        },
    ]
    
    results = []
    
    for endpoint in endpoints:
        try:
            if endpoint["method"] == "GET":
                response = requests.get(
                    endpoint["url"],
                    json=endpoint["payload"],
                    headers=HEADERS,
                    timeout=TIMEOUT
                )
            else:
                response = requests.post(
                    endpoint["url"],
                    json=endpoint["payload"],
                    headers=HEADERS,
                    timeout=TIMEOUT
                )
            
            results.append({
                "name": endpoint["name"],
                "status": response.status_code,
                "success": response.status_code in [200, 201]
            })
        except Exception as e:
            results.append({
                "name": endpoint["name"],
                "status": "Error",
                "success": False,
                "error": str(e)
            })
    
    print("\n=== Endpoint Test Summary ===")
    for result in results:
        status_icon = "✓" if result["success"] else "✗"
        print(f"{status_icon} {result['name']}: {result['status']}")
    
    return results

# Uncomment to run all tests
# test_all_endpoints()